In [10]:
import numpy as np
import pandas as pd
from pymongo import MongoClient

In [11]:
client = MongoClient('localhost', 27017)
db = client['spotify_analysis']

In [12]:
dataset = db.dataset.find({})
dataframe =  pd.DataFrame(list(dataset))

In [13]:
modified_df = dataframe
del modified_df["_id"]
del modified_df["music_name"]
del modified_df["artist"]

In [14]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()

to_normalize = ["acousticness", "danceability" ,"energy", "instrumentalness","liveness","loudness","speechiness","valence","tempo"]

for attr_to_normalize in to_normalize:
    attrb = modified_df[[attr_to_normalize]].values.astype(float)
    attrb_scaled = min_max_scaler.fit_transform(attrb)
    attrb_normalized = pd.DataFrame(attrb_scaled)
    
    del modified_df[attr_to_normalize]
    modified_df[attr_to_normalize] = attrb_normalized

In [15]:
modified_df

,first_week,second_week,third_week,fourth_week,january,february,march,april,may,june,...,position,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo
0,1,0,0,0,1,0,0,0,0,0,...,1,0.462456,0.943791,0.595825,0.000000,0.028188,0.913215,0.045969,0.881398,0.497041
1,1,0,0,0,1,0,0,0,0,0,...,2,0.594306,0.758170,0.430062,0.003952,0.073516,0.637066,0.025479,0.439043,0.448362
2,1,0,0,0,1,0,0,0,0,0,...,3,0.488826,0.552941,0.949862,0.000000,0.845121,0.851410,0.118361,0.595042,0.644396
3,1,0,0,0,1,0,0,0,0,0,...,4,0.430001,0.844444,0.592756,0.000000,0.226639,0.720417,0.015368,0.787370,0.460876
4,1,0,0,0,1,0,0,0,0,0,...,5,0.092869,0.572549,0.929397,0.000000,0.391843,0.838663,0.054462,0.864302,0.521764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199795,0,0,0,1,0,0,0,1,0,0,...,196,0.261638,0.694118,0.605034,0.004017,0.104801,0.764282,0.164195,0.463618,0.161068
199796,0,0,0,1,0,0,0,1,0,0,...,197,0.050170,0.526797,0.803540,0.019114,0.082086,0.853127,0.014020,0.383481,0.461102
199797,0,0,0,1,0,0,0,1,0,0,...,198,0.491869,0.615686,0.437225,0.000000,0.110996,0.674020,0.012402,0.113153,0.282349
199798,0,0,0,1,0,0,0,1,0,0,...,199,0.038811,0.623529,0.944746,0.000015,0.070521,0.839693,0.049339,0.077893,0.301723


In [20]:
number_of_labels = [10]
configurations = [
    (256),
    (256, 256),
    (256, 256, 256),
]

solver = "sgd"

activation_function = "logistic"

In [17]:
'''from sklearn.model_selection import train_test_split

x_train, x_test = train_test_split(modified_df, test_size=0.2, stratify=modified_df['position'])

y_train = x_train[['position']]
y_test = x_test[['position']]

del x_train['position']
del x_test['position']'''

"from sklearn.model_selection import train_test_split\n\nx_train, x_test = train_test_split(modified_df, test_size=0.2, stratify=modified_df['position'])\n\ny_train = x_train[['position']]\ny_test = x_test[['position']]\n\ndel x_train['position']\ndel x_test['position']"

In [ ]:
import json
import time
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

def define_positions(df, number_of_labels):
    new_df = df.copy()
    label_limit = 200/number_of_labels
    for i in range(number_of_labels):
        new_df.loc[(new_df['position'] > label_limit * i) & (new_df['position'] <= label_limit * (i + 1)), 'position'] = i 
    return new_df

for num_labels in number_of_labels:
    for config in configurations:
        
        new_df = define_positions(modified_df, num_labels)
        
        x_train, x_test = train_test_split(new_df, test_size=0.2, stratify=new_df['position'])

        y_train = x_train[['position']]
        y_test = x_test[['position']]

        del x_train['position']
        del x_test['position']
        
        print("Começou: " + solver + " / " + activation_function + " / " + str(config) + " / " + str(num_labels))
        
        mlp_classifier = MLPClassifier(solver = solver,activation = activation_function, hidden_layer_sizes=config, verbose=True, max_iter=10000, alpha=0.0001, tol=1e-4,  learning_rate_init=0.001)
        
        start = time.time()

        mlp_classifier.fit(x_train,y_train.values.ravel())
         
        end = time.time()
        
        time_elapsed = end - start

        training_set_score = mlp_classifier.score(x_train,y_train.values.ravel())

        test_set_score = mlp_classifier.score(x_test,y_test.values.ravel())
        
        results_obj = {
            "solver" : solver,
            "activation_function": activation_function,
            "layers" :  config,
            "last_loss" :  mlp_classifier.loss_,
            "last_iteration" :  mlp_classifier.n_iter_,
            "training_score" :  training_set_score,
            "test_score" :  test_set_score,
            "fit_time" :  time_elapsed,
            "number_of_labels" : num_labels,
            "learning_rate" : 1
        }
        
        print("Acabou: " + solver + " / " + activation_function + " / " + str(config))
        print("-----------------------------------------------")
        
        with open(f'./logs/{solver}:{activation_function}:{num_labels}:{str(config)}.result.json', 'w') as outfile:
            json.dump(results_obj, outfile)
        


Começou: sgd / logistic / 256 / 10
Iteration 1, loss = 2.30486120
Iteration 2, loss = 2.30327320
Iteration 3, loss = 2.30312348
Iteration 4, loss = 2.30303205
Iteration 5, loss = 2.30288874
Iteration 6, loss = 2.30280367
Iteration 7, loss = 2.30275442
Iteration 8, loss = 2.30247684
Iteration 9, loss = 2.30246602
Iteration 10, loss = 2.30231462
Iteration 11, loss = 2.30207426
Iteration 12, loss = 2.30204480
Iteration 13, loss = 2.30197359
Iteration 14, loss = 2.30181267
Iteration 15, loss = 2.30172142
Iteration 16, loss = 2.30152958
Iteration 17, loss = 2.30152109
Iteration 18, loss = 2.30131101
Iteration 19, loss = 2.30120848
Iteration 20, loss = 2.30099932
Iteration 21, loss = 2.30088386
Iteration 22, loss = 2.30072572
Iteration 23, loss = 2.30066896
Iteration 24, loss = 2.30047956
Iteration 25, loss = 2.30019629
Iteration 26, loss = 2.30024193
Iteration 27, loss = 2.29996083
Iteration 28, loss = 2.29984818
Iteration 29, loss = 2.29973491
Iteration 30, loss = 2.29953379
Iteration 31, 